In [1]:
print("Hello, World!")

Hello, World!


In [2]:
%pwd

'd:\\CODE\\Ai-Medical-Chatbot\\research'

In [3]:
import os
os.chdir("../")
%pwd

'd:\\CODE\\Ai-Medical-Chatbot'

In [7]:
%pip install -qU pypdf langchain-pinecone pinecone-notebooks langchain_community PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [4]:

file_path = "./Data/The-Gale-Encyclopedia-of-Medicine-3rd.pdf"  # Relative path to the file

from PyPDF2 import PdfReader

reader = PdfReader(file_path)

In [5]:
# Print metadata and content of the first page
extracted_text = ""
for page in reader.pages[:]:  # Limit to first 5 pages
    text = page.extract_text()
    if text:  # Check if text is not None
        extracted_text += text + "\n\n"

In [6]:
# Extracted text
print(len(extracted_text))

17913481


In [32]:
%pip install -qU langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,  # Small chunk size to illustrate splitting
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

# Splitting text into chunks
text_chunks = text_splitter.split_text(extracted_text)



In [9]:
# created chunks
print(len(text_chunks))

265105


In [ ]:
%pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

In [ ]:
# embedding model made
print(len(embeddings[0]))

384


In [15]:
%pip install -qU langchain-pinecone pinecone-notebooks

Note: you may need to restart the kernel to use updated packages.


In [18]:
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

In [19]:
# Import the Pinecone library
from pinecone import Pinecone

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=PINECONE_API_KEY)

In [20]:
import time

index_name = "medbot"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [25]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=model)

In [ ]:
from langchain_core.documents import Document
from langchain_pinecone import PineconeVectorStore
from langchain.embeddings import SentenceTransformerEmbeddings
from uuid import uuid4

# Convert text chunks into Document objects
documents = [Document(page_content=text) for text in text_chunks]

# Generate UUIDs
uuids = [str(uuid4()) for _ in range(len(documents))]

# Initialize LangChain's SentenceTransformer wrapper
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# Create Pinecone Vector Store with the correct embedding model
vector_store = PineconeVectorStore.from_documents(
    documents=documents,
    index_name="medbot",
    embedding=embedding_model  # Pass the LangChain-compatible embedding model
)

# Add documents to the vector store
vector_store.add_documents(documents=documents, ids=uuids)


C:\Users\Tahsin\AppData\Local\Temp\ipykernel_484\3699156783.py:13: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
